In [1]:
from bs4 import BeautifulSoup
import requests, random, time
import itertools as it
import pandas as pd

from fake_useragent import UserAgent
ua = UserAgent()

In [7]:
url = 'https://www.thecanadianencyclopedia.ca/en/browse/things/business-economics/laws-programs'

In [2]:
url = 'https://www.thecanadianencyclopedia.ca/en/browse/things'
rep = requests.get(url,headers={'User-Agent': ua.random}).text
# # lecture du code html et la recherche des balises
soup = BeautifulSoup(rep)
# récupérer tous les liens de la pages
# others_url = [_.get('href') for _ in soup.find_all("a", class_="page-link")[:-1]]


In [12]:
all_articles = []
for _ in soup.find_all('a', class_="nav-submenu-cat accordion-item dropdown-item"):
    link = _.get("href")
    if 'browse' in link:
        all_articles.append(link)

In [74]:
def get_art_content(art_url):
    # art_url = "https://www.thecanadianencyclopedia.ca/en/article/agricultural-stabilization-board"
    art_rep = requests.get(art_url,headers={'User-Agent': ua.random}).text
    art_soup = BeautifulSoup(art_rep)

    for _ in art_soup.find_all('div',id="article-content-def"):
        text = '\n'.join(p.text for p in _.find_all('p'))
    
    return text

In [34]:
def get_page_data(soup):

    #retourner le df des artciles du lien

    lks = soup.find_all("div", class_="search-single-info")
    df = {}
    df["title"] = list()
    df["link"] = list()
    df["desc"] = list()
    df["text"] = list()

    for lk in lks:
        link = lk.find("a").get('href')
        if 'article' in link:
            df["title"].append(lk.find('h3').text)
            df["link"].append(link)
            df["desc"].append(lk.find('p').text)
            df['text'].append(get_art_content(link))

    df = pd.DataFrame.from_dict(df)

    return df


In [3]:
def getAllEncyclopediaLink():

    url = 'https://www.thecanadianencyclopedia.ca/en/browse/things'
    rep = requests.get(url).text
    soup = BeautifulSoup(rep,'html.parser')

    all_articles = []
    for _ in soup.find_all('div', class_="sb-tab sb-tab--separator"):
        link = (_.find('a').get('href'))
        if len(link.split('/')) == 8:
            all_articles.append(link)

    return all_articles

In [ ]:
all_articles = getAllEncyclopediaLink()
all_articles

In [8]:
from scraping import *

In [ ]:
begin = all_articles.index('https://www.thecanadianencyclopedia.ca/en/browse/things/business-economics/companies')
rest_links = all_articles[begin:]
# rest_links

In [10]:
for idx,url in enumerate(rest_links):
    time.sleep(2)
    get_url_articles(url) 
    logging.info(f'theme {url.split("/")[-3:]} done')
    logging.info(f'Percentage on total links : {round(((idx+begin)*100)/len(all_articles),2)}%')